## Dimitrios Fikos 9/12/2024
##### Prediction Ability of competitor algorithms

The capability of a dimension reduced dataset obtained by the first $d$ principal components, to predict accurately the response variable $Y$, is assessed. We are taking a dataset where $n=5000$ and $p=12$. This dataset is splitted into the training set and the test set. The model that we are working on here is mentioned and we expect $d=2$ as its true dimension. Gradients are computed by using $k=\sqrt{n}$ and then SVD is applied to the training set. Then the first two vectors from the left singular matrix are selected. These are multiplied with the full data matrix in order to retrieve the dimension reduced dataset with dimensions ($n_{train},2$) and ($n_{test},2$) for training and test set respectively. This procedure is implemented for each of the proposed algorithms. Approach applied for the comparison is k nearest neighbors for regression. Here, we have $k=10$.  Six datasets in total are compared. The full dataset, $p=12$, the four dimension reduced datasets derived from the estimated projection vectors with $d=2$ and finally the dimension reduced dataset earned by the true projections $\eta$ and $\theta$ , again with $d=2$. The evaluation is made by the aid of Mean Squared Error (MSE) between the true response variable and the predicted one. Besides, the coefficient of determination, $R^2$ that measures the proportion of variance of the response variable explained, ranging from 0 to 1. 

In [1]:
# libraries
import math
import numpy as np
import pandas as pd
from numpy import random
from sklearn.neighbors import KDTree
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Work with Model : $$g(x)=X_1 + X_2^2 +X_3$$ 

 In this case
$$ \nabla g (x) = (1, 2X_2,1,0,\ldots, 0) ^T$$

### Create 100 random points for all algorithms

In [2]:
# Generate 100 random points N(0,1) with dimension p=12. 
rng=np.random.RandomState(42)
x_specific_train = rng.normal(loc=0, scale=1, size=(100, 12))

### Make a random dataset

In [3]:
# size n=5000 and p=12
rng=np.random.RandomState(42)
# reproducibility of data
dataset=[]
# Random variable X with 'dim' dimensions
X = rng.normal(loc=0, scale=1, size=(5000, 12))
# Make the errors
errors = rng.normal(loc=0, scale=0.7, size=5000)
# make the Y 
Y = X[:,0] + X[:, 1]**2 + X[:,2] + errors
dataset.append((X, Y))

## Train and test set split

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=30)

## Knn to the full dataset

### Writing the functions of all 4 gradient algorithms. 

#### 1) Gradient from linear regression b_ols

#### 2) Gradient from linear regression b_ols + Lasso 

#### 3) Gradient deriving from diagonal of Gram matrix b_ols_diag

#### 4) Gradient deriving from trace of Gram Matrix b_ols_trace

In [6]:
####### 1) Projection from OLS ########
def b_ols(x_specific3, X, Y, k, dim):
    estimations=[]
    # Number of neighbors
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X)
    for x in x_specific3:
        x_spec = np.array([x])  
        distances, indices = knn.kneighbors(x_spec)
        # Get the k-nearest neighbors
        nearest_X = X[indices].reshape(k, -1)
        nearest_Y = Y[indices].flatten()
        sum_xi = sum_yi =0 
        for i in range(len(nearest_X)):
            sum_xi+=nearest_X[i]
            sum_yi+=nearest_Y[i]
        sum_xi, sum_yi = sum_xi/k, sum_yi/k 
        nearest_X, nearest_Y = nearest_X - sum_xi, nearest_Y - sum_yi
        # Initialize the Linear Regression model
        linear_regression = LinearRegression(fit_intercept=False)
        # Fit the model to the data
        linear_regression.fit(nearest_X, nearest_Y)
        # Print the coefficient(s)
        estimated_gradient = linear_regression.coef_
        estimations.append(estimated_gradient)
    estimations=np.array(estimations).T
    U, S, Vh =np.linalg.svd(estimations, full_matrices=True, compute_uv=True)
    first_column = U[:, 0]
    second_column = U[:, 1]
    est_proj_1 = np.outer(first_column, first_column)
    est_proj_2 = np.outer(second_column, second_column)
    est_proj = est_proj_1 + est_proj_2
    beta_real = np.zeros((dim, 1))
    gamma_real=np.zeros((dim, 1))
    beta_real[0, 0] =  gamma_real[1,0] = beta_real[2,0] =1 
    beta_proj=beta_real@beta_real.T
    beta_proj=beta_proj*1/2
    gamma_proj=gamma_real@gamma_real.T
    true_proj=beta_proj+gamma_proj
    f_norm = np.linalg.norm(est_proj-true_proj, 'fro')
    return f_norm, U, S, Vh


####### 2) Projection deriving from OLS + Lasso ########
def b_ols_lasso(x_specific3, X, Y, k, dim):
    estimations=[]
    # Number of neighbors
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X)
    for x in x_specific3:
        x_spec = np.array([x])  
        distances, indices = knn.kneighbors(x_spec)
        # Get the k-nearest neighbors
        nearest_X = X[indices].reshape(k, -1)
        nearest_Y = Y[indices].flatten()
        sum_xi = sum_yi =0 
        for i in range(len(nearest_X)):
            sum_xi+=nearest_X[i]
            sum_yi+=nearest_Y[i]
        sum_xi, sum_yi = sum_xi/k, sum_yi/k 
        nearest_X, nearest_Y = nearest_X - sum_xi, nearest_Y - sum_yi
        # Initialize the Linear Regression model with lasso
        lasso_model = LassoCV(alphas=alphas, cv=5, fit_intercept=False, max_iter=10000, tol=1e-4)
        lasso_model.fit(nearest_X, nearest_Y)
        # Print the coefficients
        estimated_gradient = lasso_model.coef_
        estimations.append(estimated_gradient)
    estimations=np.array(estimations).T
    U, S, Vh =np.linalg.svd(estimations, full_matrices=True, compute_uv=True)
    first_column = U[:, 0]
    second_column = U[:, 1]
    est_proj_1 = np.outer(first_column, first_column)
    est_proj_2 = np.outer(second_column, second_column)
    est_proj = est_proj_1 + est_proj_2
    beta_real = np.zeros((dim, 1))
    gamma_real=np.zeros((dim, 1))
    beta_real[0, 0] =  gamma_real[1,0] = beta_real[2,0] =1 
    beta_proj=beta_real@beta_real.T
    beta_proj=beta_proj*1/2
    gamma_proj=gamma_real@gamma_real.T
    true_proj=beta_proj+gamma_proj
    f_norm = np.linalg.norm(est_proj-true_proj, 'fro')
    return f_norm, U, S, Vh

In [7]:
####### 3) Projection deriving from OLS + diagonal ######## 
def b_ols_diag(x_specific3, X, Y, k, dim):
    estimations=[]
    # Number of neighbors
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X)
    for x in x_specific3:
        x_spec = np.array([x])  
        distances, indices = knn.kneighbors(x_spec)
        # Get the k-nearest neighbors
        nearest_X = X[indices].reshape(k, -1)
        nearest_Y = Y[indices].flatten()
        b_ols=[]
        mean_X = np.mean(nearest_X, axis=0)
        mean_Y = np.mean(nearest_Y)
        for j in range(nearest_X.shape[1]):
            numer = np.mean((nearest_X[:, j] - mean_X[j]) * (nearest_Y - mean_Y))
            denom = np.mean((nearest_X[:, j] - mean_X[j]) ** 2)
            b_ols.append(numer/denom)
        estimations.append(b_ols)
    estimations=np.array(estimations).T
    U, S, Vh =np.linalg.svd(estimations, full_matrices=True, compute_uv=True)
    first_column = U[:, 0]
    second_column = U[:, 1]
    est_proj_1 = np.outer(first_column, first_column)
    est_proj_2 = np.outer(second_column, second_column)
    est_proj = est_proj_1 + est_proj_2
    beta_real = np.zeros((dim, 1))
    gamma_real=np.zeros((dim, 1))
    beta_real[0, 0] =  gamma_real[1,0] = beta_real[2,0] =1 
    beta_proj=beta_real@beta_real.T
    beta_proj=beta_proj*1/2
    gamma_proj=gamma_real@gamma_real.T
    true_proj=beta_proj+gamma_proj
    f_norm = np.linalg.norm(est_proj-true_proj, 'fro')
    return f_norm, U, S, Vh

####### 4) Projection deriving from OLS + trace ######## 
def b_ols_trace(x_specific3, X, Y, k, dim):
    estimations=[]
    # Number of neighbors
    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X)
    for x in x_specific3:
        x_spec = np.array([x])  
        distances, indices = knn.kneighbors(x_spec)
        # Get the k-nearest neighbors
        nearest_X = X[indices].reshape(k, -1)
        nearest_Y = Y[indices].flatten()
        b_ols=[]
        mean_X = np.mean(nearest_X, axis=0)
        mean_Y = np.mean(nearest_Y)
        denom=0
        for j in range(nearest_X.shape[1]):
            numer=np.mean((nearest_X[:, j] - mean_X[j]) * (nearest_Y - mean_Y))
            b_ols.append(numer)
        estimations.append(b_ols)
    estimations=np.array(estimations).T
    U, S, Vh =np.linalg.svd(estimations, full_matrices=True, compute_uv=True)
    first_column = U[:, 0]
    second_column = U[:, 1]
    est_proj_1 = np.outer(first_column, first_column)
    est_proj_2 = np.outer(second_column, second_column)
    est_proj = est_proj_1 + est_proj_2
    beta_real = np.zeros((dim, 1))
    gamma_real=np.zeros((dim, 1))
    beta_real[0, 0] =  gamma_real[1,0] = beta_real[2,0] =1 
    beta_proj=beta_real@beta_real.T
    beta_proj=beta_proj*1/2
    gamma_proj=gamma_real@gamma_real.T
    true_proj=beta_proj+gamma_proj
    f_norm = np.linalg.norm(est_proj-true_proj, 'fro')
    return f_norm, U, S, Vh

## Compute the new X_train datasets (reduced)

In [8]:
# Select the first two principal components
num_components = 2
# alphas for lasso grid
alphas = np.logspace(-6, 0, 5)

### b_ols

In [10]:
f_norm, U, S, Vh = b_ols(x_specific_train, X_train, Y_train, k=71, dim=12)  # Get singular values for this dataset
# Select the top left singular vectors corresponding to these components
top_ols= U[:,:num_components] 
# take the dimension reduced dataset
X_train_new_ols = X_train@top_ols

### b_ols_lasso

In [11]:
f_norm, U, S, Vh = b_ols_lasso(x_specific_train, X_train, Y_train, k=71, dim=12)
# Select the top left singular vectors corresponding to these components
top_lasso= U[:,:num_components]
# take the dimension reduced dataset
X_train_new_lasso = X_train@top_lasso

### b_ols_diag

In [12]:
f_norm, U, S, Vh = b_ols_diag(x_specific_train, X_train, Y_train, k=71, dim=12)
# Select the top left singular vectors corresponding to these components
top_diag = U[:,:num_components]
# take the dimension reduced dataset
X_train_new_diag = X_train@top_diag

### b_ols_trace

In [13]:
f_norm, U, S, Vh = b_ols_trace(x_specific_train, X_train, Y_train, k=71, dim=12)
# Select the top left singular vectors corresponding to these components
top_trace = U[:,:num_components]
# take the dimension reduced dataset
X_train_new_trace = X_train@top_trace

## Dimension Reduction to test set 

In [13]:
# based on the columns taken from the training set, compute the dimesnion reduced training set per each algorithm
# b_ols
X_test_new_ols = X_test@top_ols
# b_ols_lasso



In [14]:
# taking the top left singular vectors from training set, compute dimension reduced test set per algorithm
# b_ols
X_test_new_ols = X_test@top_ols
# b_ols_lasso
X_test_new_lasso = X_test@top_lasso
# b_ols_diag
X_test_new_diag = X_test@top_diag
# b_ols_trace
X_test_new_trace = X_test@top_trace

## Apply KNN regression 

In [5]:
# Train KNN in the full dataset
knn = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn.fit(X_train, Y_train)
# Predictions
Y_pred = knn.predict(X_test)
# Evaluation
mse = mean_squared_error(Y_test, Y_pred)
print(f"Mean Squared Error: {mse:.4f}")
r2 = r2_score(Y_test, Y_pred)
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 1.8707
R^2 Score: 0.5741


### b_ols

In [15]:
# Train KNN with b_ols reduced dataset
knn = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn.fit(X_train_new_ols, Y_train)
#  Make predictions on the test set
y_pred = knn.predict(X_test_new_ols)
# Evaluate the model
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 0.5940
R^2 Score: 0.8648


### b_ols_lasso

In [16]:
# Train KNN with b_ols_lasso reduced dataset
knn = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn.fit(X_train_new_lasso, Y_train)
# Make predictions on the test set
y_pred = knn.predict(X_test_new_lasso)
# Evaluate the model
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 0.5922
R^2 Score: 0.8652


### b_ols_diag

In [17]:
# Train KNN with b_ols_diag reduced dataset
knn = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn.fit(X_train_new_diag, Y_train)
# Make predictions on the test set
y_pred = knn.predict(X_test_new_diag)
# Evaluate the model
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 0.6494
R^2 Score: 0.8522


### b_ols_trace

In [18]:
# Train KNN with b_ols_trace reduced dataset 
knn = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn.fit(X_train_new_trace, Y_train)
# Make predictions on the test set
y_pred = knn.predict(X_test_new_trace)
# Evaluate the model
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 0.6288
R^2 Score: 0.8569


## Take advantage of $\eta,\theta$

In [19]:
# for the model we are using we know the ideal vectors taken for dimension reduction
eta=[1,0,1,0,0,0,0,0,0,0,0,0]
theta=[0,1,0,0,0,0,0,0,0,0,0,0]
array_U= np.column_stack((eta, theta))

In [20]:
# New train and test datasets based on η and θ 
X_train_new= X_train@array_U
X_test_new = X_test@array_U

## Application of knn regression to the true dimension reduced dataset

In [21]:
# Train KNN 
knn = KNeighborsRegressor(n_neighbors=10, weights='uniform', metric='euclidean')
knn.fit(X_train_new, Y_train)
# Make predictions on the test set
y_pred = knn.predict(X_test_new)
# Evaluate the model
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

Mean Squared Error: 0.5965
R^2 Score: 0.8642


### Comments

The full dataset shows the poorest performance with $MSE=1.87$ and $57\%$ of the explained variability. All dimension reduced datasets outperform the full dataset indicating the importance of dimension reduction in the specific model. In particular, all $R^2$ coefficients are more than $28\%$ higher than the full dataset and at the same time $MSE$ is lower. The true reduced dataset is obviously outperforming all other datasets and it is used in the experiment as a baseline, with the aim to see the four algorithms results reaching its performance and that seems to be the case here.